# Self-Refine Test 

## 1. Load environment

In [1]:
%pip install dashscope python-dotenv urllib3

Defaulting to user installation because normal site-packages is not writeable
  Using cached dashscope-1.22.2-py3-none-any.whl.metadata (6.8 kB)
Using cached dashscope-1.22.2-py3-none-any.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [74]:
import os
import dashscope
import sys
import platform
import dashscope
import time
import re
import csv
import pandas as pd
from dotenv import load_dotenv
from typing import Tuple, Optional, Dict, Union, List, Any
from dashscope.api_entities.dashscope_response import Message

In [66]:
# 1) Load environment (DASHSCOPE_API_KEY)

load_dotenv("dashscope_api_key.env", override=True)
api_key = os.getenv("DASHSCOPE_API_KEY")
if not api_key:
    print("DASHSCOPE_API_KEY not found!")

# Print environment information
print("Current environment information")
print("-" * 40)
print(f"Python version: {sys.version}")
print(f"Platform system: {platform.system()} {platform.release()}")
print(f"DASHSCOPE_API_KEY read successfully: {'✅ Yes' if api_key else '❌ No'}")

Current environment information
----------------------------------------
Python version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
Platform system: Windows 10
DASHSCOPE_API_KEY read successfully: ✅ Yes


## 2. Call Function


The **`call_model`** function is designed to invoke a large language model (LLM) through DashScope in a way that **preserves multi-round context**. Rather than passing a one-time prompt, it accepts a **list of messages**—each labeled with a role (`system`, `user`, or `assistant`)—that collectively represent the conversation history. This approach enables an iterative, dialogic style of interaction with the model:

1. **Conversation as a List of Messages**  
   - We store every piece of text that shapes the conversation (system instructions, user questions, model replies) in a Python list called `messages`.  
   - Each element is a structure with fields like `role` (e.g., `system`, `user`, `assistant`) and `content` (the actual text).

2. **System-Level Instructions**  
   - At the start of the list, we often include a “system” message that defines the model’s role or behavior (“You are a rigorous math tutor…”).  
   - Unlike a single-pass prompt, this system message persists throughout every exchange, ensuring the LLM remains consistent in style and purpose.

3. **User Queries and Model Responses**  
   - For each round of interaction, we append a new `user` message with the user’s request or instructions.  
   - The model’s reply then comes back as `assistant` text, which we likewise insert into the `messages` list.  
   - Over time, this list accumulates the entire conversation history.

4. **Self-Refine Alignment**  
   - Because all prior messages (including the model’s own previous answers) are visible in the list, the LLM can read and critique its own past output.  
   - This structure naturally supports the **feedback → refinement** cycle described in the Self-Refine paradigm, as the model sees both the problem statement and its earlier reasoning steps whenever it generates new content.

5. **DashScope Invocation**  
   - On each call, `call_model` sends the current `messages` list to the DashScope service, specifying the temperature and other parameters.  
   - DashScope’s API returns the model’s fresh reply (an “assistant” role string) and usage statistics (like token counts).  
   - We update the `messages` with the new assistant message, thus preserving context for the next round.

6. **Context Preservation**  
   - By passing **the entire conversation history** each time, we mimic the idea of a session that retains prior knowledge, enabling multi-step problem-solving or iterative refinement.  
   - This stands in contrast to single-prompt APIs that only see their immediate input. Here, the model is re-supplied with both the user’s original questions and any subsequent instructions or feedback, capturing the logic of self-critique and incremental improvement.

7. **Benefits and Considerations**  
   - **Enhanced Consistency**: The model maintains continuity across multiple rounds, remembering earlier details or constraints.  
   - **Scalability**: Because the conversation grows each iteration, users should watch out for token limit constraints and trim older or less relevant messages if needed.  
   - **Clearer Control Flow**: The code that orchestrates Self-Refine can systematically add new messages (feedback requests, draft solutions, refined outputs, etc.), making the dialogue flow explicit and easy to follow.

In [89]:
def call_model(
    messages: List[Message],
    model_name: str = "qwen2.5-math-1.5b-instruct",
    temperature: float = 0.2,
    max_retries: int = 20,
    retry_delay: float = 1.0,
    verbose: bool = False
) -> Tuple[Optional[str], Dict[str, Any]]:
    """
    Call the DashScope model with a list of messages (including system, user, assistant).

    Returns:
        content: The assistant's text response (None if error).
        usage:   usage dict (may include token counts, etc.), or {}
    """

    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key:
        print("DASHSCOPE_API_KEY not found!")
        return None, {}

    for attempt in range(1, max_retries + 1):
        try:
            response = dashscope.Generation.call(
                api_key=api_key,
                model=model_name,
                messages=messages,
                temperature=temperature,
                result_format="message",
                timeout=60
            )

            status_code = response.get("status_code", 200)  # type: ignore
            if status_code != 200:
                if verbose:
                    print(f"⚠️[Attempt {attempt}/{max_retries}] Rate limit! Retrying in {retry_delay}s...")
                    print(f"⚠️[Warning] {response.get('message', '')}  {response.get('code', '')}") # type: ignore
                time.sleep(retry_delay)
                continue

            # Parse out the model's reply
            content = response["output"]["choices"][0]["message"]["content"]  # type: ignore
            usage = response.get("usage", {})  # type: ignore
            return content, usage

        except Exception as e:
            print(f"⚠️[Attempt {attempt}/{max_retries}] Exception occurred: {e}")
            print(f"⚠️[Warning] {response.get('message', '')}  {response.get('code', '')}") # type: ignore
            time.sleep(retry_delay)

    print(f"❌ All {max_retries} attempts failed.")
    return None, {}

In [90]:
if __name__ == "__main__":
    question = "A baker makes 20 cupcakes. He eats 2 and gives away 3. How many does he have left?"
    system_prompt = (
        "You are a rigorous math tutor. "
        "You solve problems step by step in detail and ensure mathematical correctness. "
        "Then, at the end, on a new line, write:\n"
        "Final Answer: <numeric result>\n"
        "Do not include any additional text after that line."
    )
    messages = [
        Message(role="system", content=system_prompt),
        Message(role="user", content=f"Solve the following problem step-by-step:\n{question}")
    ]

    content, usage = call_model(
        messages=messages,
        model_name="qwen2.5-math-1.5b-instruct",
        temperature=0.2,
        verbose=True
    )

    print("\n=== Model Output ===")
    print("Assistant Response:\n", content)
    print("Usage:", usage)


=== Model Output ===
Assistant Response:
 To determine how many cupcakes the baker has left, we need to follow these steps:

1. Start with the initial number of cupcakes the baker has, which is 20.
2. Subtract the number of cupcakes the baker eats, which is 2.
3. Subtract the number of cupcakes the baker gives away, which is 3.

Let's perform the calculations step-by-step:

1. Initial number of cupcakes: 20
2. After eating 2 cupcakes: \(20 - 2 = 18\)
3. After giving away 3 cupcakes: \(18 - 3 = 15\)

So, the number of cupcakes the baker has left is \(\boxed{15}\).
Usage: {"input_tokens": 97, "output_tokens": 149, "total_tokens": 246, "cached_tokens": 0}


## 3. Extract Answer

In [91]:
def extract_final_answer(response_text: str) -> str:
    """
    Extract the final numeric or concise answer from the given response text.

    The order of priority is as follows:
    1. Check "Final Answer: ..." (case-insensitive).
    2. Check the last occurrence of "\boxed{...}" (LaTeX style). From that box, parse advanced numeric forms.
    3. If no box found or no valid numeric in it, scan the entire text for the last recognized numeric form.
    4. If nothing is found at all, return the entire trimmed response text.
    """

    # ----------------------------
    # 1) Check for "Final Answer: ..."
    #    (Now has the highest priority)
    # ----------------------------
    fa_match = re.search(r"(?i)final answer\s*:\s*([^\n]+)", response_text)
    if fa_match:
        candidate = fa_match.group(1).strip()
        # If we want to parse advanced numeric forms from this candidate,
        # we can do so here. Otherwise, we return directly.
        return candidate

    # ----------------------------
    # 2) Check for the last occurrence of "\boxed{...}"
    #    We'll parse advanced numeric forms from within it.
    #    If multiple boxes exist, we take the last one.
    # ----------------------------
    # Regex to match all \boxed{...} blocks (including nested braces is tricky, but we do a simple approach)
    all_boxes = re.findall(r"\\boxed\{([^}]*)\}", response_text, flags=re.DOTALL)
    if all_boxes:
        box_content = all_boxes[-1]  # take the last box
        # Try to parse advanced numeric forms from the box content
        numbers_in_box = parse_numeric_expressions(box_content)
        if numbers_in_box:
            # If multiple numbers found, return the last one
            return numbers_in_box[-1].strip()
        # If no recognized numeric, return the raw content
        return box_content.strip()

    # ----------------------------
    # 3) Fallback: parse the entire text for recognized numeric forms
    #    Return the last one if available.
    # ----------------------------
    all_numbers = parse_numeric_expressions(response_text)
    if all_numbers:
        return all_numbers[-1].strip()

    # ----------------------------
    # 4) Final fallback: return entire string
    # ----------------------------
    return response_text.strip()


def parse_numeric_expressions(text: str):
    """
    Parse various numeric expressions from the text, including:
    - Optional sign: [+/-]
    - Integers or decimals (e.g., 123, -3.14)
    - Simple LaTeX fraction: \frac{number}{number}

    Returns a list of all matched string forms in order of appearance.
    """
    # We'll combine a few patterns:
    # 1) signed integer or decimal: [+-]?\d+(?:\.\d+)?
    # 2) simple fraction form: \frac\s*\{?\s*[+-]?\d+(\.\d+)?\s*\}?\s*/\s*\{?\s*[+-]?\d+(\.\d+)?\s*\}?
    pattern = r"""
        [+-]?\d+(?:\.\d+)?              # e.g. -3, 4.5, +2.0
        |
        \\frac\s*\{?\s*[+-]?\d+(?:\.\d+)?\s*\}?\s*/\s*\{?\s*[+-]?\d+(?:\.\d+)?\s*\}?  # e.g. \frac{3}{5}, \frac{-2.5}{10}
    """

    # Using VERBOSE flag to allow inline comments
    matches = re.findall(pattern, text, flags=re.VERBOSE)
    return matches

## 4. Self-Refine

The `self_refine` function implements a multi-round refinement process for math problem solving, following the Self-Refine paradigm. It uses a single, persistent `messages` list to simulate a continuous dialogue with the model, allowing it to build upon its own previous answers and feedback.

### Key Steps:

1. **Initial Draft**
   - The function begins by prompting the model to generate a step-by-step solution to the math problem.
   - This initial response is saved as an assistant message in the session.

2. **Feedback and Refinement Loop**
   - Each round, the model is asked to critique its last answer.
   - It then receives its own feedback and is prompted to revise accordingly.
   - All exchanges (questions, answers, feedback) are appended to the same `messages` list, preserving full context.

3. **Session-Based Memory**
   - By keeping the entire conversation in a single session, the model sees the full reasoning history at every step.
   - This allows coherent self-correction and aligns with the original Self-Refine paper’s methodology.

4. **Early Stopping (with Safeguard)**
   - The model may exit early if it declares the answer already correct — but only after completing at least **two** refinement cycle.
   - This ensures robustness without redundant computation.

5. **Final Output**
   - The last refined answer is parsed to extract the final numeric result.
   - The function returns this result, the full answer text, number of rounds used, and total token usage.

### Key Points & Considerations

- **Rigorous System Prompt**  
  - The model is cast in the role of a “rigorous math tutor,” ensuring detailed reasoning and clarity in every generation.
  - >Since both temperature and top_p can control the diversity of generated text, it is recommended that you only set one of them. 
    - So I lowered the temperature a bit, reduced the diversity appropriately, and ensured the rigor of the mathematical results.

- **Feedback-Refine Loop**  
  The model alternates between critiquing its own output and improving it, thereby aiming to catch logical or computational mistakes in a structured manner.

- **Mandatory Improvement Round**  
  The function will not accept an immediate “no improvement” response for the first refinement attempt. This ensures that, at a minimum, the model has made one pass to reevaluate its initial draft.

- **Early Stop Heuristic**  
  After at least one refinement, if the model states that no further changes are needed, the loop terminates early. This method is a convenience to reduce overhead when the model is sufficiently confident.

- **No Absolute Guarantee of Correctness**  
  While the approach often enhances solution quality, the model’s own judgment may still fail to catch subtler errors. For high-stakes scenarios, external validation (e.g., test suites or domain experts) is recommended.

- **Return Structure**  
  The final output includes:
  1. **`final_text`**: The model’s last refined draft (full textual answer).  
  2. **`final_answer`**: The numeric or short result extracted from `final_text`.  
  3. **`ended_on_round`**: Which round triggered the exit, either because of reaching the maximum iteration or early stopping.  
  4. **`total_usage`**: A cumulative usage metric (e.g., total tokens).

This procedure illustrates a practical way to nudge large language models toward more self-critical and iteratively improved answers, particularly for math-oriented tasks.

In [92]:
def self_refine(
    question: str,
    rounds: int = 5,
    verbose: bool = True,
    model_name: str = "qwen2.5-math-1.5b-instruct",
    temperature: float = 0.2,
) -> Dict:
    """
    Conduct multiple rounds of 'Self-Refine' for a math problem in a single session (messages list),
    returning the final refined answer.
    
    Args:
        question (str): The math problem to be solved.
        rounds (int): The maximum number of refinement iterations.
        verbose (bool): Whether to print intermediate steps.
        model_name (str): The name of the language model to call.
        temperature (float): Sampling temperature for controlling randomness in the output.

    Returns:
        dict with keys:
          {
              "final_text"      : str or None, 
              "final_answer"    : str or None,
              "ended_on_round"  : int,
              "total_usage"     : int  # or float, depending on actual usage measure
          }
    """

    # -----------------------------
    # 1) Construct an initial 'messages' list
    # -----------------------------
    # (A) system prompt
    system_content = (
        "You are a rigorous math tutor. "
        "You solve problems step by step in detail and ensure mathematical correctness. "
        "When you finish, on a new line, write:\n"
        "Final Answer: <numeric result>\n"
        "Do not include any additional text after that line."
    )
    messages = [Message(role="system", content=system_content)]

    # (B) user: problem statement
    #   This is akin to the old "first_prompt_template"
    user_intro = (
        "Solve the following math problem with detailed reasoning. "
        "Explain your steps clearly and logically. "
        "At the end, please write:\n"
        "Final Answer: <number>\n\n"
        f"{question}"
    )
    messages.append(Message(role="user", content=user_intro))
    
    total_usage_tokens = 0
    ended_on_round = 0

    # -----------------------------
    # 2) Generate the initial draft (assistant)
    # -----------------------------
    initial_draft, usage_init = call_model(
        messages=messages,                # <--- We pass the entire messages so far
        model_name=model_name,
        temperature=temperature,
        verbose=verbose
    )
    if usage_init is None:
        usage_init = {"total_tokens": 0}
    total_usage_tokens += usage_init["total_tokens"]

    if initial_draft is None:
        if verbose:
            print("❌[Error] Initial draft generation returned None.")
        return {
            "final_text"    : None,
            "final_answer"  : None,
            "ended_on_round": 0,
            "total_usage"   : total_usage_tokens
        }

    # Save the assistant's response to messages
    messages.append(Message(role="assistant", content=initial_draft))

    if verbose:
        print("\n✅━━ [Initial Draft Generated] ━━✅")
        print(initial_draft)
        print("")

    current_answer = initial_draft

    # -----------------------------
    # 3) Iterative refinement
    # -----------------------------
    for i in range(rounds):
        round_index = i + 1
        if verbose:
            print(f"[Refinement Round {round_index} of {rounds}]")

        # 3a) user asks for feedback
        feedback_request = (
            f"Below is the math problem and your draft solution. Please analyze it:\n\n"
            f"Problem:\n{question}\n\n"
            f"Draft Answer:\n{current_answer}\n\n"
            "Point out any errors or omissions, suggest improvements. "
            "If it's already correct, say 'no improvement needed'."
        )
        messages.append(Message(role="user", content=feedback_request))

        feedback, usage_fb = call_model(
            messages=messages,
            model_name=model_name,
            temperature=temperature,
            verbose=verbose
        )
        if usage_fb is None:
            usage_fb = {"total_tokens": 0}
        total_usage_tokens += usage_fb["total_tokens"]

        if feedback is None:
            if verbose:
                print("❌[Error] Feedback generation returned None. Stopping.")
            ended_on_round = round_index
            break

        # Save model's feedback to messages (assistant role)
        messages.append(Message(role="assistant", content=feedback))

        if verbose:
            print("✅[Feedback Received]")
            print(feedback)
            print("")

        # 3b) check if "no improvement" (but only if round_index > 1 to ensure at least 1 round of refine)
        lower_fb = feedback.lower()
        if round_index > 2 and (
            "no improvement" in lower_fb
            or "solution is correct" in lower_fb
            or "already correct" in lower_fb
        ):
            if verbose:
                print("✅[Notice] Feedback indicates no further improvement is needed. Stopping early.")
            ended_on_round = round_index
            break

        # 3c) user instructs to refine
        refine_request = (
            f"Here is the math problem, your draft solution, and the feedback:\n\n"
            f"Problem:\n{question}\n\n"
            f"Draft Answer:\n{current_answer}\n\n"
            f"Feedback:\n{feedback}\n\n"
            "Please refine the draft solution. "
            "At the end, write 'Final Answer: <number>'. "
            "Make sure the numeric result is correct."
        )
        messages.append(Message(role="user", content=refine_request))

        refined_answer, usage_refine = call_model(
            messages=messages,
            model_name=model_name,
            temperature=temperature,
            verbose=verbose
        )
        if usage_refine is None:
            usage_refine = {"total_tokens": 0}
        total_usage_tokens += usage_refine["total_tokens"]

        if refined_answer is None:
            if verbose:
                print("⚠️[Warning] Refinement returned None. Using current version and stopping.")
            ended_on_round = round_index
            break

        # Save new refined answer as assistant
        messages.append(Message(role="assistant", content=refined_answer))
        current_answer = refined_answer

        if verbose:
            print("✅[Refined Answer]")
            print(current_answer)
            print("")

        ended_on_round = round_index

    # -----------------------------
    # 4) Extract final numeric answer
    # -----------------------------
    final_answer = extract_final_answer(current_answer)

    # -----------------------------
    # 5) Return
    # -----------------------------
    return {
        "final_text"    : current_answer,
        "final_answer"  : final_answer,
        "ended_on_round": ended_on_round,
        "total_usage"   : total_usage_tokens
    }

## 5. Single Demo Question Test

In [93]:
if __name__ == "__main__":
    question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

    # Gold answer
    gold_answer = "18"

    # Do self-consistency
    result = self_refine(
        question=question,
        model_name="qwen2.5-math-1.5b-instruct",
        rounds=3
    )
    final_text = result["final_text"]
    final_answer = result["final_answer"]
    ended_on_round = result["ended_on_round"]
    total_usage = result["total_usage"]

    # Compare with gold
    is_correct = (final_answer.strip() == gold_answer.strip())
    response_length = len(final_text)

    
    # Print the info
    print("\n━━━ ★  SINGLE QUESTION TEST  ★ ━━━")
    print(f"📌  Question:\n{question}\n")
    print(f"💡  Predicted Final Answer: {final_answer}")
    print(f"🎯  Gold Answer: {gold_answer}")
    print(f"✅  Correct? {is_correct}")
    print(f"🔎  Response Length (chars): {response_length}")
    print(f"🌀  Ended on Round: {ended_on_round}")
    print(f"📊  Total Usage (tokens): {total_usage}")
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")


✅━━ [Initial Draft Generated] ━━✅
To determine how much Janet makes every day at the farmers' market, we need to follow these steps:

1. Calculate the total number of eggs laid by the ducks each day.
2. Determine how many eggs Janet sells at the farmers' market each day.
3. Calculate the revenue from selling the eggs at the farmers' market.

**Step 1: Calculate the total number of eggs laid by the ducks each day.**

Janet's ducks lay 16 eggs per day.

**Step 2: Determine how many eggs Janet sells at the farmers' market each day.**

Janet eats 3 eggs for breakfast every morning and bakes 4 eggs for her friends every day. Therefore, the total number of eggs she does not sell is:
\[ 3 + 4 = 7 \text{ eggs} \]

The number of eggs Janet sells at the farmers' market each day is:
\[ 16 - 7 = 9 \text{ eggs} \]

**Step 3: Calculate the revenue from selling the eggs at the farmers' market.**

Janet sells each egg for $2. Therefore, the revenue from selling 9 eggs is:
\[ 9 \times 2 = 18 \text{ do

## 7. GSM8K main_test 

In [ ]:
# =============================
# Your helper functions
# =============================
def extract_numeric(ans: str) -> str:
    """Clean extracted answer by removing $, units, commas, and extra tokens."""
    ans = ans.strip()
    ans = re.sub(r"[^0-9.\-]", "", ans)
    return ans

def extract_gold_answer(gold_text: str) -> str:
    """Extract the numeric answer from GSM8K-style '#### <number>' line."""
    match = re.search(r"####\s*(\d+(?:\.\d+)?)", str(gold_text))
    if match:
        return match.group(1)
    numbers = re.findall(r"\d+(?:\.\d+)?", str(gold_text))
    if numbers:
        return numbers[-1]
    return str(gold_text).strip()

def is_correct_with_tolerance(pred: str, gold: str, epsilon: float = 1e-5) -> bool:
    """Compare pred and gold within a tolerable margin of error."""
    try:
        return abs(float(pred) - float(gold)) < epsilon
    except ValueError:
        return pred.strip() == gold.strip()

def is_abnormal_answer(ans: str, max_repeat: int = 10, max_len: int = 100) -> bool:
    ans = ans.strip()
    # Empty or overly long numeric string
    if len(ans) > max_len:
        return True
    # Check for repeated characters like 777777...
    if re.match(r"^(\d)\1{" + str(max_repeat) + r",}$", ans):
        return True
    return False

# =============================
# (Import or define) self_refine
# Here we assume you have the single-session version of self_refine
# or any correct version that returns a dict with keys:
#  "final_text", "final_answer", "ended_on_round", "total_usage"
# =============================
# from your_module import self_refine

# =============================
# Load GSM8K CSV dataset
# =============================
df = pd.read_csv("dataset/GSM8K/main_test.csv")
num_samples = -1  # -1 for processing the full dataset; or set a smaller number for debugging
subset = df if num_samples == -1 else df.head(num_samples)

save_path = "results/results_gsm8k_self_refine_qwen.csv"
done_indices = set()

# If we want to resume from an existing CSV, load it and skip done indices
if os.path.exists(save_path):
    try:
        existing_df = pd.read_csv(save_path)
        done_indices = set(existing_df["index"].tolist())
        print(f"🔁 Resuming from existing results in '{save_path}'. Rows done: {len(done_indices)}")
    except Exception as e:
        print(f"⚠️ Could not read existing file: {e}")

# =============================
# Main Evaluation Loop
# =============================
for idx, row in subset.iterrows():
    if idx in done_indices:
        continue

    question = row["question"]
    gold_answer_raw = row["answer"]  # original gold text, e.g. "#### 15"

    print(f"\n=== Processing index {idx} ===")
    print("Question:", question)

    # 1) Extract gold numeric
    gold_clean = extract_gold_answer(gold_answer_raw)

    # 2) Call self_refine
    try:
        refine_result = self_refine(
            question=question,
            rounds=3,  # or your preferred number
            verbose=False,
            model_name="qwen2.5-math-1.5b-instruct",  # or any model you want
            temperature=0.2
        )
        final_text = refine_result["final_text"]
        predicted_answer = refine_result["final_answer"] or ""
        ended_on_round = refine_result["ended_on_round"]
        total_usage = refine_result["total_usage"]
    except Exception as e:
        print(f"[Error] {e}")
        final_text = ""
        predicted_answer = ""
        ended_on_round = 0
        total_usage = 0

    # 3) Clean the predicted answer
    pred_clean = extract_numeric(predicted_answer)
    if is_abnormal_answer(pred_clean):
        print("⚠️ Detected abnormal numeric pattern, marking as INVALID.")
        pred_clean = "INVALID"

    # 4) Check correctness within tolerance
    correct = False
    if pred_clean != "INVALID":
        correct = is_correct_with_tolerance(pred_clean, gold_clean)

    response_length = len(final_text)

    # 5) Print for inspection
    print("Gold Extracted:", repr(gold_clean))
    print("Predicted:", repr(pred_clean))
    print("Correct?", correct)
    print("Ended On Round:", ended_on_round)
    print("Usage Tokens:", total_usage)

    # 6) Save result to CSV
    row_dict = {
        "index": idx,
        "question": question,
        "gold_answer": gold_answer_raw,
        "gold_clean": gold_clean,
        "predicted_answer": pred_clean,
        "correct": correct,
        "response_length": response_length,
        "ended_on_round": ended_on_round,
        "total_usage": total_usage
    }

    write_header = not os.path.exists(save_path)
    with open(save_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=row_dict.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(row_dict)

    print(f"✅Saved result for index {idx}")

print("\nAll done!")


=== Processing index 0 ===
Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Gold Extracted: '18'
Predicted: '18'
Correct? True
Ended On Round: 2
Usage Tokens: 4558
✅Saved result for index 0

=== Processing index 1 ===
Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
Gold Extracted: '3'
Predicted: '3'
Correct? True
Ended On Round: 2
Usage Tokens: 3106
✅Saved result for index 1

=== Processing index 2 ===
Question: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
Gold Extracted: '70000'
Predicted: '120000'
Correct? False
Ended On Round: 2
Usage Tokens: 

KeyboardInterrupt: 